# Generating Gamma Surfaces & Stacking Fault

Gamma surfaces (also called generalised stacking faults) and stacking faults are forms of crystal defects formed by sliding two slabs of a crystal against each other along a given plane. Often, these defects have metastable states (also called stacking faults), which can exist at grain boundaries or between two partial dislocations. Here, we deal with the full pathway from a complete crystal, sliding through any metastable points until we return back to the perfect crystal again. 

For a stacking fault, the slabs are moved along a single line, whereas for gamma surfaces this is the full plane (hence generalised stacking fault). `matscipy.gamma_surface` provides two classes - `GammaSurface` and `StackingFault` - in order to model these systems, including utility methods for relaxing and plotting the associated potential energy surfaces.

## Stacking Faults for a Specific Dislocation
When modelling systems of dissociated dislocations connected by a stacking fault, it can be useful to see the stacking fault in isolation. To do this, we will use `matscipy.gamma_surface.StackingFault` to model the stacking fault associated with the dissociated `matscipy.dislocations.DiamondGlideScrew` dislocation in Si:

In [1]:
from matscipy.dislocation import DiamondGlideScrew, get_elastic_constants
from matscipy.gamma_surface import StackingFault
from matscipy.calculators.manybody.explicit_forms.stillinger_weber import StillingerWeber,\
                                                                Holland_Marder_PRL_80_746_Si
from matscipy.calculators.manybody import Manybody
from visualisation import show_dislocation, interactive_view
from ase.visualize import view

calc = Manybody(**StillingerWeber(Holland_Marder_PRL_80_746_Si))

# the function accepts any ASE type of calculator
alat, C11, C12, C44 = get_elastic_constants(calculator=calc, symbol="Si")
print(f"alat = {alat}")
Si_screw = DiamondGlideScrew(alat, C11, C12, C44, symbol="Si")

Si_screw_bulk, Si_screw_dislo = Si_screw.build_cylinder(radius=20, partial_distance=5)

show_dislocation(Si_screw_dislo, 
                 diamond_structure=True, # bonds make it a bit easier to see the structure
                 scale=0.3, add_bonds=True, 
                 partial_distance=5 * Si_screw.glide_distance, 
                 d_name="1/6<112> 30 degree partial screw") 


      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 17:26:22      -34.692786*       0.0922
FIRE:    1 17:26:22      -34.692742*       0.1881
FIRE:    2 17:26:22      -34.692797*       0.0452
FIRE:    3 17:26:22      -34.692771*       0.1323
FIRE:    4 17:26:22      -34.692781*       0.1071
FIRE:    5 17:26:22      -34.692794*       0.0616
FIRE:    6 17:26:22      -34.692800*       0.0043
FIRE:    7 17:26:22      -34.692795*       0.0537
FIRE:    8 17:26:22      -34.692796*       0.0511
FIRE:    9 17:26:22      -34.692796*       0.0461
FIRE:   10 17:26:22      -34.692798*       0.0390
FIRE:   11 17:26:22      -34.692799*       0.0299
FIRE:   12 17:26:22      -34.692799*       0.0195
FIRE:   13 17:26:22      -34.692800*       0.0081
FIRE:   14 17:26:22      -34.692800*       0.0036
FIRE:   15 17:26:22      -34.692800*       0.0036
FIRE:   16 17:26:22      -34.692800*       0.0035
FIRE:   17 17:26:22      -34.692800*       0.0034
FI

NGLWidget()

To model just the stacking fault, we can call `StackingFault` with just the lattice constant, crystalstructure (one of "fcc", "bcc", "diamond"), the chemical symbol, and the kind of dissociated dislocation, and use the function `StackingFault.generate_images()` to generate the fault images.

In [5]:
fault = StackingFault(alat, DiamondGlideScrew, symbol="Si")
fault.generate_images(n=32) # Generate 32 images along the sliding pathway
images = [image * (1, 1, 2) for image in fault.images]
cell = images[0].cell[:, :]
for image in images:
    image.set_cell(cell)
    image.wrap()
view(images, viewer="ngl")